**Summary**:
The nested schema isnt as intimidating as it looks. The biggest thing that looks scary is the `quoted_status` coloumn which just looks like it contains all the information relating to a quoted tweet. For now we can ignore this. 

There are around 657 307 tweets. At level one. Not sure if we explode the tweet quotes we'll get more unique quotes.

There are only 14 007 tweets with a geo point attached to them. This is very small and insignificant. What saves us through is that tweets have been attached to places! with the `place` col. This gives us a geo location box that we can fit some one into. Have a look at the `place` section for more information relating to the counts. 

We can also have tweets already catagorised into languages and have hashtags neatly structured for us. We just have to extract it correctly. See the `hash tags` section for what the counts look like.

# Start Spark

In [28]:
pd.set_option("max_colwidth",10000)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, desc
from pyspark.sql.types import ArrayType, StringType, MapType, FloatType

In [2]:
spark = SparkSession.builder\
                        .master("local[*]")\
                        .config('spark.executor.memory', '5g')\
                        .config('spark.driver.memory', '5g')\
                        .config("spark.sql.session.timeZone", "UTC")\
                        .config("spark.sql.execution.arrow.enabled","false")\
                    .appName("Sentiment Analysis")\
                    .getOrCreate()
spark

In [3]:
data=spark.read.json("../../data/geotagged_tweets_20160812-0912.jsons")

In [62]:
data.count()

657307

# Schema Of Everything

In [4]:
data.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true

# Look at first level of json and some geo stuff

In [5]:
from pyspark.sql.functions import col

In [33]:
data.select(
    "created_at",
    "favorite_count",
    "geo.coordinates",
    "geo.type",
    "text",
    "lang",
    "retweet_count",
    "retweeted",
    "source",
    "timestamp_ms"
      ).filter(col("geo.coordinates").isNotNull()).limit(5).toPandas()

,created_at,favorite_count,coordinates,type,text,lang,retweet_count,retweeted,source,timestamp_ms
0,Fri Aug 12 10:15:04 +0000 2016,0,"[1.3656, 103.8277]",Point,⒍ #Trump\n⒎ #IKON\n⒏ Europe\n⒐ Korean\n⒑ Japan\n\n2016/8/12 18:09 SGT #trndnl https://t.co/psP0GzBgZB,in,0,False,"<a href=""http://trendinalia.com"" rel=""nofollow"">Trendinalia Singapore</a>",1470996904679
1,Fri Aug 12 10:26:24 +0000 2016,0,"[66.160507, -153.369141]",Point,@realDonaldTrump It's time to boycott Socialist news network CNN. Sad. #MakeAmericaGreatAgain https://t.co/L92W1GIPRy,en,0,False,"<a href=""http://www.keksec.org"" rel=""nofollow"">Twitter Helper v3.0</a>",1470997584097
2,Fri Aug 12 10:35:04 +0000 2016,0,"[1.3656, 103.8277]",Point,⒍ #Trump\n⒎ #ISIS\n⒏ Europe\n⒐ Korean\n⒑ Japan\n\n2016/8/12 18:29 SGT #trndnl https://t.co/psP0GzBgZB,in,0,False,"<a href=""http://trendinalia.com"" rel=""nofollow"">Trendinalia Singapore</a>",1470998104714
3,Fri Aug 12 10:49:37 +0000 2016,0,"[42.47631904, -83.0024906]",Point,"Hey #ImWithHer🇺🇸👠, and we're making ""Herstory"" together because we're #StrongerTogether!… https://t.co/HE2lYITEfb",en,0,False,"<a href=""http://instagram.com"" rel=""nofollow"">Instagram</a>",1470998977365
4,Fri Aug 12 10:55:02 +0000 2016,0,"[1.3656, 103.8277]",Point,⒍ #Trump\n⒎ #ISIS\n⒏ Europe\n⒐ Korean\n⒑ Jimin\n\n2016/8/12 18:49 SGT #trndnl https://t.co/psP0GzBgZB,in,0,False,"<a href=""http://trendinalia.com"" rel=""nofollow"">Trendinalia Singapore</a>",1470999302800


In [16]:
data.select("geo.type").groupBy("type").count().show()

+-----+------+
| type| count|
+-----+------+
| null|643300|
|Point| 14007|
+-----+------+



In [23]:
data.groupBy("lang").count().show()

+----+------+
|lang| count|
+----+------+
|  en|563329|
|  ne|     9|
|  vi|    10|
|  ro|   152|
|  sl|    22|
| und| 72178|
|  ur|     7|
|  lv|    37|
|  pl|   161|
|  hy|     2|
|  pt|   537|
|  tl|  1013|
|  in|   702|
|  ko|    12|
|  uk|     4|
|  cs|    69|
|  mr|     2|
|  tr|   260|
|  de|   793|
|  is|    34|
+----+------+
only showing top 20 rows



In [7]:
data.count()

657307

# Look at `place col`

In [67]:
data.select("place.country","place.country_code","place.full_name","place.name","place.place_type","place.url","place.bounding_box.coordinates")\
.limit(5).toPandas()

,country,country_code,full_name,name,place_type,url,coordinates
0,United States,US,"Frontenac, MO",Frontenac,city,https://api.twitter.com/1.1/geo/id/29a119f18820c3ad.json,"[[[-90.433909, 38.606163], [-90.433909, 38.647891], [-90.405462, 38.647891], [-90.405462, 38.606163]]]"
1,United States,US,"Baton Rouge, LA",Baton Rouge,city,https://api.twitter.com/1.1/geo/id/c7ef5f3368b68777.json,"[[[-91.218994, 30.354161], [-91.218994, 30.564039], [-90.962605, 30.564039], [-90.962605, 30.354161]]]"
2,United States,US,"Frontenac, MO",Frontenac,city,https://api.twitter.com/1.1/geo/id/29a119f18820c3ad.json,"[[[-90.433909, 38.606163], [-90.433909, 38.647891], [-90.405462, 38.647891], [-90.405462, 38.606163]]]"
3,Australia,AU,"Melbourne, Victoria",Melbourne,city,https://api.twitter.com/1.1/geo/id/01864a8a64df9dc4.json,"[[[144.593742, -38.433859], [144.593742, -37.511274], [145.512529, -37.511274], [145.512529, -38.433859]]]"
4,United States,US,"Baltimore, MD",Baltimore,city,https://api.twitter.com/1.1/geo/id/c0b8e8dc81930292.json,"[[[-76.711521, 39.197211], [-76.711521, 39.372215], [-76.529443, 39.372215], [-76.529443, 39.197211]]]"


In [69]:
data.select("place.full_name").groupBy("full_name").count().sort(desc("count")).show()

+-------------------+-----+
|          full_name|count|
+-------------------+-----+
|       Florida, USA|17582|
|    Los Angeles, CA|12917|
|  Pennsylvania, USA|12640|
|      Manhattan, NY|12423|
|       Georgia, USA|10121|
|        Chicago, IL| 9880|
|      Kentucky, USA| 9119|
|North Carolina, USA| 7443|
|      New York, USA| 7290|
|         Texas, USA| 7218|
|        Houston, TX| 6279|
|      Virginia, USA| 5770|
|       Brooklyn, NY| 5748|
|     Louisiana, USA| 5146|
|    California, USA| 5076|
|         Austin, TX| 4998|
|    New Jersey, USA| 4739|
|          Ohio, USA| 4675|
|     Washington, DC| 4415|
|        Phoenix, AZ| 4191|
+-------------------+-----+
only showing top 20 rows



In [70]:
data.select("place.place_type").groupBy("place_type").count().sort(desc("count")).show()

+------------+------+
|  place_type| count|
+------------+------+
|        city|489275|
|       admin|156311|
|     country|  8082|
|         poi|  2218|
|neighborhood|  1390|
|        null|    31|
+------------+------+



In [45]:
data.groupBy("place.country").count().sort(desc("count")).show()

+--------------+------+
|       country| count|
+--------------+------+
| United States|591990|
|        Canada| 17228|
|United Kingdom|  8599|
|        México|  7830|
|     Australia|  2613|
|        Mexico|  2439|
|         India|  1593|
|        France|  1257|
|Estados Unidos|  1041|
|       Ireland|   915|
|     Singapore|   906|
|      Colombia|   804|
|       Germany|   800|
|         Japan|   646|
|  South Africa|   633|
|   Deutschland|   623|
|     Nederland|   561|
|        Italia|   559|
|       Vietnam|   531|
|         Spain|   497|
+--------------+------+
only showing top 20 rows



`place.url` is useless

In [27]:
import pandas as pd

# Look at `user col`

In [36]:
# a first look of the nested json thats contained in user
data.select("user").limit(5).toPandas()

,user
0,"(False, Fri Sep 02 14:54:17 +0000 2011, False, False, Fashion photographer, love action and adventure, care for the less fortunate, don't tolerate any kind of racism regardless of race or religion, 1708, None, 186, None, 446, True, 366636488, 366636488, False, en, 19, Miami, GIL DUPUY, None, 131516, http://abs.twimg.com/images/themes/theme14/bg.gif, https://abs.twimg.com/images/themes/theme14/bg.gif, True, None, http://pbs.twimg.com/profile_images/760915612910497792/bz4WtvbZ_normal.jpg, https://pbs.twimg.com/profile_images/760915612910497792/bz4WtvbZ_normal.jpg, 009999, EEEEEE, EFEFEF, 333333, True, False, DUPUY77, 17620, None, http://ggm-dupuy.com, None, False)"
1,"(False, Thu Oct 15 00:28:04 +0000 2009, False, False, AWSCWI Pipefitter USAF NRA I remember USA, Hunting Fishing, HarleyDavidsons- a time when we HS students could carry guns in our trks 2 school, 3850, None, 531, None, 677, True, 82496193, 82496193, False, en, 26, Ogletown, DE, Red Octopus, None, 000000, http://abs.twimg.com/images/themes/theme9/bg.gif, https://abs.twimg.com/images/themes/theme9/bg.gif, False, https://pbs.twimg.com/profile_banners/82496193/1453903674, http://pbs.twimg.com/profile_images/688374640360685568/xcSrhPsj_normal.jpg, https://pbs.twimg.com/profile_images/688374640360685568/xcSrhPsj_normal.jpg, 3B94D9, 000000, 000000, 000000, False, False, redoctapus, 5046, Eastern Time (US & Canada), None, -14400, False)"
2,"(False, Fri Sep 02 14:54:17 +0000 2011, False, False, Fashion photographer, love action and adventure, care for the less fortunate, don't tolerate any kind of racism regardless of race or religion, 1708, None, 186, None, 446, True, 366636488, 366636488, False, en, 19, Miami, GIL DUPUY, None, 131516, http://abs.twimg.com/images/themes/theme14/bg.gif, https://abs.twimg.com/images/themes/theme14/bg.gif, True, None, http://pbs.twimg.com/profile_images/760915612910497792/bz4WtvbZ_normal.jpg, https://pbs.twimg.com/profile_images/760915612910497792/bz4WtvbZ_normal.jpg, 009999, EEEEEE, EFEFEF, 333333, True, False, DUPUY77, 17621, None, http://ggm-dupuy.com, None, False)"
3,"(False, Tue Jun 02 01:59:59 +0000 2009, False, False, The best movies are written and directed by people with a natural mental illness, 1514, None, 180, None, 298, True, 44032624, 44032624, False, en, 7, Australia, JanJorgenson, None, 022330, http://pbs.twimg.com/profile_background_images/587088433538936832/T7xq1e0m.jpg, https://pbs.twimg.com/profile_background_images/587088433538936832/T7xq1e0m.jpg, True, https://pbs.twimg.com/profile_banners/44032624/1418695714, http://pbs.twimg.com/profile_images/720017763683557376/nB_KossR_normal.jpg, https://pbs.twimg.com/profile_images/720017763683557376/nB_KossR_normal.jpg, 94D487, FFFFFF, 99CC33, 3E4415, True, False, jjorgi30, 2231, Melbourne, None, 36000, False)"
4,"(False, Mon Aug 20 09:43:48 +0000 2012, False, False, Retired, political wonk mother of one grandmother of 2, democrat moderate progressive, 141, None, 2652, None, 2976, True, 769208504, 769208504, False, en, 84, None, Beverly Spence, None, EBEBEB, http://abs.twimg.com/images/themes/theme7/bg.gif, https://abs.twimg.com/images/themes/theme7/bg.gif, False, None, http://pbs.twimg.com/profile_images/3108377856/76f3f0220b457f0a264358f1f807f6f9_normal.jpeg, https://pbs.twimg.com/profile_images/3108377856/76f3f0220b457f0a264358f1f807f6f9_normal.jpeg, 990000, DFDFDF, F3F3F3, 333333, True, False, bspence5, 92605, None, None, None, False)"


In [66]:
data.select("user.name",
            "user.geo_enabled",
            "user.following",
            "user.followers_count",
            "created_at",
            "user.verified")\
            .limit(5).toPandas()

,name,geo_enabled,following,followers_count,created_at,verified
0,GIL DUPUY,True,None,186,Fri Aug 12 10:04:00 +0000 2016,False
1,Red Octopus,True,None,531,Fri Aug 12 10:04:02 +0000 2016,False
2,GIL DUPUY,True,None,186,Fri Aug 12 10:04:10 +0000 2016,False
3,JanJorgenson,True,None,180,Fri Aug 12 10:04:21 +0000 2016,False
4,Beverly Spence,True,None,2652,Fri Aug 12 10:04:30 +0000 2016,False


**Its interesting that coordinates is a 4 point location** This is really usefull for us to pinpoint people.

# hashtags

In [55]:
from pyspark.sql.functions import explode, lit

In [58]:
hashtags = data.select(explode("entities.hashtags.text").alias("hashtag"),lit(1)).groupBy("hashtag").count().sort(desc("count")).toPandas()

In [60]:
hashtags.head(20)

,hashtag,count
0,Trump,46847
1,MAGA,17067
2,NeverTrump,16615
3,ImWithHer,15426
4,TrumpPence16,12924
5,Hillary,12456
6,HillaryClinton,12161
7,NeverHillary,12099
8,DonaldTrump,10505
9,CrookedHillary,10061
